In [1]:
from models.resevit_road import restevit_road_cls
from torchsummary import summary
import torch
from fvcore.nn import FlopCountAnalysis,flop_count_table
from Load_data import Torch_load
from torch import nn
from tqdm import tqdm
from torch import optim
from Load_data.KTorch import load_test 
from Report_model.Report_Torch import Class_Report

In [2]:
model=restevit_road_cls()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model= model.to(device)
summary(model, (3,224,224), batch_size=1)
inputs= torch.rand(1,3, 224,224)
inputs=inputs.to(device)
flops=FlopCountAnalysis(model,inputs)
flops.total()
flop_count_table(flops)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 64, 112, 112]           9,408
       BatchNorm2d-2          [1, 64, 112, 112]             128
              ReLU-3          [1, 64, 112, 112]               0
         MaxPool2d-4            [1, 64, 56, 56]               0
         resnet_in-5            [1, 64, 56, 56]               0
            Conv2d-6          [1, 16, 112, 112]             432
       BatchNorm2d-7          [1, 16, 112, 112]              32
         Hardswish-8          [1, 16, 112, 112]               0
         ConvLayer-9          [1, 16, 112, 112]               0
           Conv2d-10          [1, 16, 112, 112]             144
      BatchNorm2d-11          [1, 16, 112, 112]              32
        Hardswish-12          [1, 16, 112, 112]               0
        ConvLayer-13          [1, 16, 112, 112]               0
           Conv2d-14          [1, 16, 1

Unsupported operator aten::add_ encountered 82 time(s)
Unsupported operator aten::max_pool2d encountered 1 time(s)
Unsupported operator aten::hardswish encountered 30 time(s)
Unsupported operator aten::add encountered 41 time(s)
Unsupported operator aten::mul encountered 15 time(s)
Unsupported operator aten::pad encountered 7 time(s)
Unsupported operator aten::div encountered 7 time(s)
Unsupported operator aten::softmax encountered 8 time(s)
The following submodules of the model were never called during the trace of the graph. They may be unused, or they were accessed by direct calls to .forward() or via other python methods. In the latter case they will have zeros for statistics, though their statistics will still contribute to their parent calling module.
base_mdoel.efficientvit_in.input_stem.op_list.1.shortcut, base_mdoel.stage1.stage.op_list.1.shortcut, base_mdoel.stage2.stage.op_list.1.shortcut, base_mdoel.stage2.stage.op_list.2.shortcut, base_mdoel.stage3.stage.op_list.1.context_

'| module                                                  | #parameters or shape   | #flops     |\n|:--------------------------------------------------------|:-----------------------|:-----------|\n| model                                                   | 17.04M                 | 2.516G     |\n|  base_mdoel                                             |  17.038M               |  2.516G    |\n|   base_mdoel.resnet_in                                  |   9.536K               |   0.122G   |\n|    base_mdoel.resnet_in.conv                            |    9.408K              |    0.118G  |\n|    base_mdoel.resnet_in.bn                              |    0.128K              |    4.014M  |\n|   base_mdoel.efficientvit_in.input_stem.op_list         |   0.928K               |   13.447M  |\n|    base_mdoel.efficientvit_in.input_stem.op_list.0      |    0.464K              |    6.423M  |\n|    base_mdoel.efficientvit_in.input_stem.op_list.1.main |    0.464K              |    7.025M  |\n|   base_

In [3]:
def save_checkpoint(model, filename):
    torch.save(model.state_dict(), filename)
    
def train(epochs_per_fold=25,batch_size=32,image_size=224):
    checkpoint = torch.load('ResEViT_model.pth')
    model.load_state_dict(checkpoint)
    for fold,(dataloader_dict) in enumerate(Torch_load(image_size=image_size,batch_size=batch_size)()):
        print("Fold:",fold+1)
        criterior= nn.CrossEntropyLoss()
        if fold<1: optimizer = optim.Adam(model.parameters(), lr=0.0001)
        elif fold<3: optimizer = optim.Adam(model.parameters(), lr=0.00001)
        else: optimizer = optim.Adam(model.parameters(), lr=0.000001)
        early_stop_thresh = 5
        best_accuracy = -1
        best_epoch = -1
        stop=False
        for epoch in range(epochs_per_fold):
            if stop==True: break
            with open('output2.txt', 'a') as f:
                print("Epoch {}/{}".format(epoch, epochs_per_fold),file=f)
            for phase in ["Train", "Val"]:
                if phase == "Train":
                    model.train()
                else:
                    model.eval()
                    
                epoch_loss = 0.0
                epoch_corrects = 0
                
                for inputs, labels in tqdm(dataloader_dict[phase]):
                    inputs = inputs.to(device)
                    labels = labels.to(device)
                    optimizer.zero_grad()
                    
                    with torch.set_grad_enabled(phase == "Train"):
                        outputs = model(inputs)
                        loss = criterior(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                        
                        if phase == "Train":
                            loss.backward()
                            optimizer.step()
                            
                    epoch_loss += loss.item()*inputs.size(0)
                    epoch_corrects += torch.sum(preds==labels.data)
                epoch_loss = epoch_loss / len(dataloader_dict[phase].dataset)
                epoch_accuracy = epoch_corrects.double() / len(dataloader_dict[phase].dataset)
                if phase == "Val":
                    if epoch_accuracy > best_accuracy:
                        best_accuracy = epoch_accuracy
                        best_epoch = epoch
                        save_checkpoint(model, "ResEViT_model.pth")
                    elif epoch - best_epoch > early_stop_thresh:
                        print("Early stopped training at epoch %d" % epoch)
                        stop=True  # terminate the training loop
                        
                with open('output2.txt', 'a') as f:
                    print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_accuracy), file=f)

In [4]:
train()

Fold: 1


100%|██████████| 140/140 [00:50<00:00,  2.79it/s]


Fold: 2


 65%|██████▍   | 361/558 [02:40<01:27,  2.24it/s]


KeyboardInterrupt: 

In [4]:
checkpoint = torch.load('ResEViT_model.pth')
model.load_state_dict(checkpoint)
# torch.save(model, './ResEViT_model')
# model = torch.load('./ResEViT_model')
Test_data=load_test(16,224)
Class_Report(Test_data,model,device)()

100%|██████████| 108/108 [00:24<00:00,  4.38it/s]

              precision    recall  f1-score   support

         0.0     0.8571    0.8854    0.8710       942
         1.0     0.8552    0.8211    0.8378       777

    accuracy                         0.8563      1719
   macro avg     0.8562    0.8532    0.8544      1719
weighted avg     0.8563    0.8563    0.8560      1719

